In [1]:
from pathlib import Path
import json
import re
import pandas as pd

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os



In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from pathlib import Path
import json

folder_path = "Atlas dataset"
pdf_loader = PyPDFDirectoryLoader(folder_path)
raw_docs = pdf_loader.load()



In [3]:
raw_docs

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2026-01-05T23:04:38+01:00', 'author': '', 'moddate': '2026-01-05T23:04:38+01:00', 'title': 'Microsoft Word - Document2', 'source': 'Atlas dataset\\City-guides.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='A. New York City Guide - GrandStay ProperƟes \nDocument ID: GSH-EXT-NYC-001 \nLast Updated: March 20, 2024 \nApplicable: All NYC properƟes \nTOP ATTRACTIONS: \n1. Must-See Landmarks: \no Statue of Liberty & Ellis Island \n\uf0a7 Ferry from BaƩery Park: $24.50 adults \n\uf0a7 Advance Ɵckets required (book 3+ months ahead) \n\uf0a7 First ferry: 8:30 AM, last return: 5:00 PM \no Empire State Building \n\uf0a7 Hours: 8:00 AM - 2:00 AM daily \n\uf0a7 Tickets: $44 standard, $77 express \n\uf0a7 Best Ɵmes: Early morning or aŌer 10:00 PM \no Metropolitan Museum of Art \n\uf0a7 Hours: Sunday-Thursday 10:00 AM - 5:00 PM, Friday-Saturday 10:00 AM - \n9:00 PM \n\uf0a7 Suggested a

In [4]:
from langchain.schema import Document
def clean_pdf_text(text: str) -> str:
    text = re.sub(r"\uf0a7", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"Page \d+", "", text)
    return text.strip()

cleaned_docs = [
    Document(
        page_content=clean_pdf_text(doc.page_content),
        metadata=doc.metadata
    )
    for doc in raw_docs
]


In [5]:
cleaned_docs

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2026-01-05T23:04:38+01:00', 'author': '', 'moddate': '2026-01-05T23:04:38+01:00', 'title': 'Microsoft Word - Document2', 'source': 'Atlas dataset\\City-guides.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='A. New York City Guide - GrandStay ProperƟes Document ID: GSH-EXT-NYC-001 Last Updated: March 20, 2024 Applicable: All NYC properƟes TOP ATTRACTIONS: 1. Must-See Landmarks: o Statue of Liberty & Ellis Island Ferry from BaƩery Park: $24.50 adults Advance Ɵckets required (book 3+ months ahead) First ferry: 8:30 AM, last return: 5:00 PM o Empire State Building Hours: 8:00 AM - 2:00 AM daily Tickets: $44 standard, $77 express Best Ɵmes: Early morning or aŌer 10:00 PM o Metropolitan Museum of Art Hours: Sunday-Thursday 10:00 AM - 5:00 PM, Friday-Saturday 10:00 AM - 9:00 PM Suggested admission: $30 adults Free for NYC residents & students 2. Neighborhood Highlights: o Times

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(cleaned_docs)


In [7]:
chunks

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2026-01-05T23:04:38+01:00', 'author': '', 'moddate': '2026-01-05T23:04:38+01:00', 'title': 'Microsoft Word - Document2', 'source': 'Atlas dataset\\City-guides.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='A. New York City Guide - GrandStay ProperƟes Document ID: GSH-EXT-NYC-001 Last Updated: March 20, 2024 Applicable: All NYC properƟes TOP ATTRACTIONS: 1. Must-See Landmarks: o Statue of Liberty & Ellis Island Ferry from BaƩery Park: $24.50 adults Advance Ɵckets required (book 3+ months ahead) First ferry: 8:30 AM, last return: 5:00 PM o Empire State Building Hours: 8:00 AM - 2:00 AM daily Tickets: $44 standard, $77 express Best Ɵmes: Early morning or aŌer 10:00 PM o Metropolitan Museum of Art Hours: Sunday-Thursday 10:00 AM - 5:00 PM, Friday-Saturday 10:00 AM - 9:00 PM Suggested admission: $30'),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'Py

In [8]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [9]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [10]:
embeddings = OpenAIEmbeddings()


C:\Users\HP\AppData\Local\Temp\ipykernel_14000\1734200243.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [11]:
text = "how are you"
text = embeddings.embed_query(text)

In [12]:
len(text)

1536

In [13]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "aurora"

if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine"
    )

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)


In [14]:
docsearch.similarity_search("Statue of Liberty ferry price", k=2)


[Document(id='f0acae0a-ecfc-4ba1-b83c-a8e1e5693d8f', metadata={'author': '', 'creationdate': '2026-01-05T23:04:38+01:00', 'creator': 'PyPDF', 'moddate': '2026-01-05T23:04:38+01:00', 'page': 0.0, 'page_label': '1', 'producer': 'Microsoft: Print To PDF', 'source': 'Aurora dataset\\City-guides.pdf', 'title': 'Microsoft Word - Document2', 'total_pages': 6.0}, page_content='A. New York City Guide - GrandStay ProperƟes Document ID: GSH-EXT-NYC-001 Last Updated: March 20, 2024 Applicable: All NYC properƟes TOP ATTRACTIONS: 1. Must-See Landmarks: o Statue of Liberty & Ellis Island Ferry from BaƩery Park: $24.50 adults Advance Ɵckets required (book 3+ months ahead) First ferry: 8:30 AM, last return: 5:00 PM o Empire State Building Hours: 8:00 AM - 2:00 AM daily Tickets: $44 standard, $77 express Best Ɵmes: Early morning or aŌer 10:00 PM o Metropolitan Museum of Art Hours: Sunday-Thursday 10:00 AM - 5:00 PM, Friday-Saturday 10:00 AM - 9:00 PM Suggested admission: $30'),
 Document(id='2faa3ed0-c3

In [15]:
from langchain.prompts import PromptTemplate

policy_prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "guest_type",
        "loyalty",
        "city"
    ],
    template="""
You are a POLICY INTERPRETATION AGENT for Atlas Horizon Hospitality.

ROLE
- Interpret hotel policies strictly and conservatively
- Use ONLY the retrieved policy text
- Never infer or soften policy language

CONTEXT
Guest Type: {guest_type}
Loyalty Tier: {loyalty}
City: {city}

POLICY DOCUMENTS
{context}

USER QUESTION
{question}

OUTPUT FORMAT (JSON ONLY):
{{
  "policy_facts": [string],
  "limitations": [string],
  "applicability": string,
  "confidence": number
}}
"""
)


In [16]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [17]:


from langchain.chains import ConversationalRetrievalChain

policy_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": policy_prompt},
    return_source_documents=False
)


In [18]:
chat_history = []

query = "Hola—is breakfast included? Also wifi speed?"

result = policy_chain.invoke({
    "question": query,
    "chat_history": chat_history,
    "guest_type": "Leisure",
    "loyalty": "Gold",
    "city": "Sydney"
})
chat_history.append((query, result["answer"]))
print(result["answer"])


```json
{
  "policy_facts": [],
  "limitations": [],
  "applicability": "The provided policy documents do not include information regarding breakfast inclusion.",
  "confidence": 1
}
```


## Agentic Rag Implementation

In [19]:
# Load conversation data
import json

conversation_file_path = "Atlas dataset/gsh_support_chats_jan_jun_2024.json"
with open(conversation_file_path, 'r', encoding='utf-8') as f:
    conversation_data = json.load(f)

print(f"Loaded {len(conversation_data)} conversations")

Loaded 50 conversations


In [20]:
from langchain.schema import Document

conversation_docs = []

for conv in conversation_data:
    # Extract key information
    guest_messages = [m['text'] for m in conv['messages'] if m['sender'] == 'guest']
    assistant_messages = [m['text'] for m in conv['messages'] if m['sender'] == 'assistant']
    
    # Get grounding sources used in this conversation
    grounding_sources = []
    for msg in conv['messages']:
        if msg.get('grounding_source_type') and msg['grounding_source_type'] != 'NONE':
            grounding_sources.append(msg['grounding_source_type'])
    
    # Convert to set to remove duplicates, then back to list for slicing
    unique_grounding_sources = list(set(grounding_sources))
    
    # Create document content
    doc_content = f"""
CONVERSATION ID: {conv['conversation_id']}
INTENT: {conv['intent_primary']}
SECONDARY INTENT: {conv['intent_secondary'] if conv['intent_secondary'] else 'None'}
BRAND: {conv['brand']}
CITY: {conv['property']['city']}
COUNTRY: {conv['property']['country']}
GUEST TYPE: {conv['guest_profile']['guest_type']}
LOYALTY TIER: {conv['guest_profile']['loyalty_tier']}
RESOLUTION STATUS: {conv['resolution']['status']}
CSAT SCORE: {conv['resolution']['csat_score']}
TAGS: {', '.join(conv['tags'])}

KEY GUEST QUESTIONS:
{chr(10).join([f"- {msg}" for msg in guest_messages[:3]])}

SUCCESSFUL ASSISTANT RESPONSES:
{chr(10).join([f"- {msg}" for msg in assistant_messages[:3]])}

GROUNDING SOURCES USED: {', '.join(unique_grounding_sources) if unique_grounding_sources else 'None'}

CONVERSATION SUMMARY:
This conversation shows how to handle a {conv['intent_primary']} inquiry for a {conv['guest_profile']['guest_type']} traveler 
with {conv['guest_profile']['loyalty_tier']} status in {conv['property']['city']}. The assistant used sources: 
{', '.join(unique_grounding_sources[:2]) if unique_grounding_sources else 'general knowledge'}.
"""
    
    conversation_docs.append(Document(
        page_content=doc_content.strip(),
        metadata={
            "type": "conversation",
            "conversation_id": conv['conversation_id'],
            "intent": conv['intent_primary'],
            "secondary_intent": conv['intent_secondary'],
            "city": conv['property']['city'],
            "country": conv['property']['country'],
            "brand": conv['brand'],
            "guest_type": conv['guest_profile']['guest_type'],
            "loyalty_tier": conv['guest_profile']['loyalty_tier'],
            "tags": str(conv['tags']),
            "csat_score": conv['resolution']['csat_score'],
            "source": "conversation_history"
        }
    ))

print(f"Created {len(conversation_docs)} conversation documents")

Created 50 conversation documents


In [21]:

# Create separate index for conversations
conversation_index_name = "aurora-conversations"

# Clean metadata to handle null values
cleaned_conversation_docs = []
for doc in conversation_docs:
    # Create a new metadata dictionary with cleaned values
    cleaned_metadata = {}
    for key, value in doc.metadata.items():
        # Convert null/None to empty string or appropriate value
        if value is None:
            cleaned_metadata[key] = ""
        elif isinstance(value, list):
            cleaned_metadata[key] = str(value)  # Convert lists to string
        elif isinstance(value, dict):
            cleaned_metadata[key] = str(value)  # Convert dicts to string
        else:
            cleaned_metadata[key] = value
    
    # Create new document with cleaned metadata
    cleaned_doc = Document(
        page_content=doc.page_content,
        metadata=cleaned_metadata
    )
    cleaned_conversation_docs.append(cleaned_doc)

print(f"Cleaned {len(cleaned_conversation_docs)} conversation documents")

# Store conversation documents
conversation_vectorstore = PineconeVectorStore.from_documents(
    documents=cleaned_conversation_docs,
    embedding=embeddings,
    index_name=conversation_index_name
)



Cleaned 50 conversation documents


In [22]:
conversation_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an agent that analyses historical conversations, read through how the agent handled similiar questions in the past,
and answer your USER QUESTION based on that., what you need to do is to check your historical conversations read through the conversations and answer the 
user based on that question.

ROLE
- Analyze how similar guest questions were historically handled
- Focus on tone, escalation patterns, uncertainty handling
- also get how the questions were resolved by analyzing the conservations relating to the question
- DO NOT invent policy

HISTORICAL CONVERSATIONS
{context}

USER QUESTION
{question}

OUTPUT FORMAT (JSON ONLY):
{{
  "observed_patterns": string,
  "response_style": string,
  "conversation": string,
  "confidence": number
}}
"""
)


In [23]:
conversation_retriever = conversation_vectorstore.as_retriever()

In [24]:
from langchain.chains import RetrievalQA

conversation_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=conversation_retriever,
    chain_type_kwargs={"prompt": conversation_prompt},
    return_source_documents=False
)


In [25]:
from langchain.tools import tool

@tool
def policy_agent_tool(
    question: str,
    guest_type: str,
    loyalty: str,
    city: str
):
    """
    Use this tool to interpret official hotel policies.
    It retrieves and summarizes authoritative policy documents
    relevant to the guest question, city, loyalty tier, and guest type.
    The output reflects strict policy rules and limitations.
    """
    return policy_chain.invoke({
        "question": question,
        "chat_history": [],
        "guest_type": guest_type,
        "loyalty": loyalty,
        "city": city
    })["answer"]


In [26]:

from langchain.tools import tool

@tool
def conversation_agent_tool(question: str):
    """
    Use this tool to analyze historical guest–assistant conversations.
    It identifies how similar questions were previously handled,
    including tone, uncertainty handling, escalation patterns,
    and typical assistant responses. This tool does NOT interpret policy
    and should be used to inform response style and guest experience only.
    """
    return conversation_chain.invoke({"query": question})["result"]


In [27]:
import asyncio

In [28]:
async def run_agents_parallel(
    question,
    guest_type,
    loyalty,
    city
):
    policy_task = asyncio.to_thread(
        policy_agent_tool.invoke,
        {
            "question": question,
            "guest_type": guest_type,
            "loyalty": loyalty,
            "city": city
        }
    )

    conversation_task = asyncio.to_thread(
        conversation_agent_tool.invoke,
        {
            "question": question
        }
    )

    policy_result, conversation_result = await asyncio.gather(
        policy_task,
        conversation_task
    )

    return policy_result, conversation_result


In [29]:
aggregator_prompt = PromptTemplate(
    input_variables=[
        "policy_output",
        "conversation_output",
        "question"
    ],
    template="""
    You are the FINAL RESPONSE AGENT for Atlas Horizon.
    Your role is to generate the best possible customer-facing answer by combining policy rules with real historical agent–guest conversations.
    ensure that your final answer is in english.
    CORE PRINCIPLES

    Policy output defines what is allowed and disallowed. Never violate it.

    Conversation output provides real-world context, explanations, exceptions, and phrasing that may not be explicitly stated in the policy.

    If the conversation output contains additional details not present in the policy, you may use them only if they do not contradict policy.

    When policy is unclear or confidence is below 0.7, adopt careful, conditional language.

    If conversation and policy differ, policy always overrides facts, but conversation can still guide tone and explanation depth.

    If ambiguity remains, recommend confirming with support or the property.

    INPUTS

    Policy Agent Output (authoritative rules):
    {policy_output}

    Conversation Agent Output (how similar questions were answered in real customer-care interactions):
    {conversation_output}

    User Question:
    {question}

    TASK

    Synthesize a final response that:

    Is factually compliant with policy

    Reflects how agents have historically explained or handled this situation

    Sounds natural, human, and helpful

    Includes clarifications or caveats when needed

    FINAL ANSWER REQUIREMENTS

    Length: 2–4 sentences

    Tone: clear, calm, customer-friendly

    Do not mention internal agents, policies, or confidence scores

    If uncertainty exists, gently suggest confirmation

    FINAL ANSWER:

    """
)


In [30]:
from langchain.chains import LLMChain

aggregator_chain = LLMChain(
    llm=llm,
    prompt=aggregator_prompt
)


C:\Users\HP\AppData\Local\Temp\ipykernel_14000\1168021799.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  aggregator_chain = LLMChain(


In [31]:
async def agentic_rag_answer(
    question,
    guest_type,
    loyalty,
    city
):
    policy_output, conversation_output = await run_agents_parallel(
        question,
        guest_type,
        loyalty,
        city
    )

    final_answer = aggregator_chain.invoke({
        "policy_output": policy_output,
        "conversation_output": conversation_output,
        "question": question
    })

    return final_answer


In [32]:
query = "What time is breakfast served?"

response = await agentic_rag_answer(
    question=query,
    guest_type="Business",
    loyalty="Gold",
    city="Sydney"
)

print(response)


{'policy_output': '{\n  "policy_facts": [\n    "Breakfast is served from 6:30 AM to 10:30 AM on weekdays.",\n    "Breakfast is served from 7:00 AM to 11:00 AM on weekends."\n  ],\n  "limitations": [],\n  "applicability": "Applicable to all guests.",\n  "confidence": 1\n}', 'conversation_output': '{\n  "observed_patterns": "The assistant consistently provides clear and concise information regarding breakfast hours, specifying different times for weekdays and weekends. It also offers to confirm any seasonal adjustments, which shows attentiveness to the guest\'s needs.",\n  "response_style": "The tone is professional and informative, addressing the guest\'s question directly while maintaining a friendly demeanor. The assistant ensures to provide additional context when necessary.",\n  "conversation": "Breakfast is typically served 6:30–10:30 AM on weekdays and 7:00–11:00 AM on weekends. If your property has seasonal adjustments, I can confirm for this location.",\n  "confidence": 0.95\n}'

## ADDING CONVERSATION MEMORY

In [33]:
import sqlite3

db = sqlite3.connect("conversation_memory.db", check_same_thread=False)

db.execute("""
CREATE TABLE IF NOT EXISTS conversation_memory (
    session_id TEXT,
    role TEXT,
    message TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")

db.commit()


In [34]:
def store_memory(session_id, role, message):
    db.execute(
        "INSERT INTO conversation_memory (session_id, role, message) VALUES (?, ?, ?)",
        (session_id, role, message)
    )
    db.commit()


def get_conversation_memory(session_id, limit=6):
    rows = db.execute(
        """
        SELECT role, message
        FROM conversation_memory
        WHERE session_id = ?
        ORDER BY timestamp DESC
        LIMIT ?
        """,
        (session_id, limit)
    ).fetchall()

    return "\n".join(
        f"{role}: {msg}" for role, msg in reversed(rows)
    )


In [35]:
def get_chat_history_tuples(session_id, limit=6):
    rows = db.execute(
        """
        SELECT role, message
        FROM conversation_memory
        WHERE session_id = ?
        ORDER BY timestamp ASC
        """,
        (session_id,)
    ).fetchall()

    history = []
    user_msg = None

    for role, msg in rows:
        if role == "user":
            user_msg = msg
        elif role == "assistant" and user_msg:
            history.append((user_msg, msg))
            user_msg = None

    return history[-limit:]


In [36]:
def get_chat_history_text(session_id, limit=6):
    rows = db.execute(
        """
        SELECT role, message
        FROM conversation_memory
        WHERE session_id = ?
        ORDER BY timestamp DESC
        LIMIT ?
        """,
        (session_id, limit)
    ).fetchall()

    return "\n".join(
        f"{role}: {msg}" for role, msg in reversed(rows)
    )


In [37]:
@tool
def policy_agent_tool(
    question: str,
    guest_type: str,
    loyalty: str,
    city: str,
    chat_history: list
):
    """
    Use this tool to interpret official hotel policies.
    It retrieves and summarizes authoritative policy documents
    relevant to the guest question, city, loyalty tier, and guest type.
    The output reflects strict policy rules and limitations.
    """
    return policy_chain.invoke({
        "question": question,
        "chat_history": chat_history,
        "guest_type": guest_type,
        "loyalty": loyalty,
        "city": city
    })["answer"]


In [38]:
@tool
def conversation_agent_tool(
    question: str,
    conversation_memory: str
):
    """
    Use this tool to analyze historical guest–assistant conversations.
    It identifies how similar questions were previously handled,
    including tone, uncertainty handling, escalation patterns,
    and typical assistant responses. This tool does NOT interpret policy
    and should be used to inform response style and guest experience only.
    """
    return conversation_chain.invoke({
        "query": question,
        "chat_history": conversation_memory
    })["result"]


In [39]:
async def run_agents_parallel(
    question,
    guest_type,
    loyalty,
    city,
    chat_history_tuples,
    chat_history_text
):
    policy_task = asyncio.to_thread(
        policy_agent_tool.run,
        {
            "question": question,
            "guest_type": guest_type,
            "loyalty": loyalty,
            "city": city,
            "chat_history": chat_history_tuples
        }
    )

    conversation_task = asyncio.to_thread(
        conversation_agent_tool.run,
        {
            "question": question,
            "conversation_memory": chat_history_text
        }
    )

    return await asyncio.gather(policy_task, conversation_task)


In [40]:
async def agentic_rag_answer(
    question,
    guest_type,
    loyalty,
    city,
    session_id
):
    chat_history_tuples = get_chat_history_tuples(session_id)
    chat_history_text = get_chat_history_text(session_id)

    policy_output, conversation_output = await run_agents_parallel(
        question,
        guest_type,
        loyalty,
        city,
        chat_history_tuples,
        chat_history_text
    )

    final_answer = aggregator_chain.invoke({
        "policy_output": policy_output,
        "conversation_output": conversation_output,
        "conversation_memory": chat_history_text,
        "question": question
    })

    store_memory(session_id, "user", question)
    store_memory(session_id, "assistant", final_answer["text"])

    return final_answer


In [45]:
session_id = "guest_002"

response = await agentic_rag_answer(
    question="What time is breakfast served?",
    guest_type="Business",
    loyalty="Gold",
    city="Sydney",
    session_id=session_id
)

print(response["text"])


Breakfast is served from 6:30 AM to 10:30 AM on weekdays and from 7:00 AM to 11:00 AM on weekends. If you're staying at our Sydney location, please let me know if you have any specific questions or if you'd like me to confirm any seasonal adjustments to the schedule. Enjoy your stay!


In [46]:
response = await agentic_rag_answer(
    question="Is it included with my booking?",
    guest_type="Business",
    loyalty="Gold",
    city="Sydney",
    session_id=session_id
)

print(response["text"])


Breakfast availability depends on your specific booking rate and package. While breakfast is served in the Aurelia Restaurant from 6:30 AM to 10:30 AM, I recommend checking your reservation details or confirming with the property to see if it is included with your stay. If you have any other questions or need further assistance, feel free to ask!


In [49]:
response = await agentic_rag_answer(
    question="Hello! Do you have airport shuttle at Rome?",
    guest_type="Business",
    loyalty="Gold",
    city="Sydney",
    session_id=session_id
)

print(response)

{'policy_output': '```json\n{\n  "policy_facts": [],\n  "limitations": ["Airport transfers: $65 one-way"],\n  "applicability": "The policy does not mention an airport shuttle service in Rome.",\n  "confidence": 1\n}\n```', 'conversation_output': '{\n  "observed_patterns": "The assistant typically confirms the availability of an airport shuttle, clarifies that some properties may offer scheduled shuttles while others may partner with car services, and requests additional details such as the arrival airport and landing time to provide tailored advice.",\n  "response_style": "The tone is friendly and informative, aiming to provide clear and helpful information while encouraging further engagement by asking for more details.",\n  "conversation": "Some GSH properties offer a scheduled shuttle, while others partner with vetted car services. If you share your arrival airport and approximate landing time, I can advise the best option for this property.",\n  "confidence": 0.95\n}', 'conversatio